# 利用cuBLAS库处理第2级GEMV运算
矩阵向量乘法

我们生成1个10*100的随机矩阵A和一个大小为100的随机向量x，然后把y初始化成由10个0元素组成的向量，最后将alpha设置为1，把beta设置为0，这样设置的目的是直接进行矩阵相乘而无需缩放。

In [1]:
import pycuda.autoinit
from pycuda import gpuarray
import numpy as np
from skcuda import cublas
from time import time

m = 10
n = 100
alpha = 1
beta = 0
A = np.random.rand(m, n).astype('float32')
x = np.random.rand(n).astype('float32')
y= np.zeros(m).astype('float32')

A_columnwise = A.T.copy()
A_gpu = gpuarray.to_gpu(A_columnwise)
x_gpu = gpuarray.to_gpu(x)
y_gpu = gpuarray.to_gpu(y)

trans = cublas._CUBLAS_OP['N']

lda = m
incx = 1
incy = 1
handle = cublas.cublasCreate()

# y <- alpha A x + beta y
# handle是句柄，trans是指定的矩阵结构，默认矩阵按列主序格式进行存储的，m和n是A的行数和列数，alpha表示alpha的值，A是矩阵，lda是主维度 x是与矩阵相乘的向量，incx是步长，beta是beta的值，
cublas.cublasSgemv(handle, trans, m, n, alpha, A_gpu.gpudata, lda, x_gpu.gpudata, incx, beta, y_gpu.gpudata, incy)

cublas.cublasDestroy(handle)
print('cuBLAS returned the correct value: %s' % np.allclose(np.dot(A, x), y_gpu.get()))

/home/makerlab/anaconda3/envs/pycuda/lib/python3.8/site-packages/skcuda/cublas.py:284: UserWarning: creating CUBLAS context to get version number
  warnings.warn('creating CUBLAS context to get version number')


cuBLAS returned the correct value: True
